# Harsh Sharma

## Big Data Deliverable (Dec 5, 2022)

---

Insatll spark and make sure you change the kernel to PySpark.

In [1]:
# import spark
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1670016207855_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Load the dataset

In [2]:
# Read in some data locally
df = spark.read.csv('s3://brainstation-dsft/eng_1M_1gram.csv', header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# let's see how the dataset looks like
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----+---------+-----+-----+
|    token|year|frequency|pages|books|
+---------+----+---------+-----+-----+
|inGermany|1927|        2|    2|    2|
|inGermany|1929|        1|    1|    1|
|inGermany|1930|        1|    1|    1|
|inGermany|1933|        1|    1|    1|
|inGermany|1934|        1|    1|    1|
|inGermany|1935|        1|    1|    1|
|inGermany|1938|        5|    5|    5|
|inGermany|1939|        1|    1|    1|
|inGermany|1940|        1|    1|    1|
|inGermany|1942|        2|    2|    2|
|inGermany|1944|        2|    2|    2|
|inGermany|1946|        2|    2|    2|
|inGermany|1947|        3|    3|    2|
|inGermany|1948|        1|    1|    1|
|inGermany|1949|        1|    1|    1|
|inGermany|1952|        1|    1|    1|
|inGermany|1956|        1|    1|    1|
|inGermany|1957|        2|    2|    2|
|inGermany|1959|        1|    1|    1|
|inGermany|1960|        3|    3|    3|
+---------+----+---------+-----+-----+
only showing top 20 rows

Whoo!! Pretty short in terms of features....

#### 4.a Describe the dataset (examples include size, shape, schema) in pyspark

Let's see the number of samples we have in the dataset.

In [4]:
# checking the number of rows we have
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

261823225

WooooW!! quite big, 261 MILLION.

In [30]:
# Get columns count
cols = len(df.columns)
print(f"DataFrame Columns count : {cols}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame Columns count : 5

In [15]:
# peeking into the columns of the df
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- token: string (nullable = true)
 |-- year: string (nullable = true)
 |-- frequency: string (nullable = true)
 |-- pages: string (nullable = true)
 |-- books: string (nullable = true)

From above we can see that there are 5 columns in the given dataset.

All the columns in the dataset are of string/object dtype. But it can be clearly seen that year, frequency, pages and books are in numbers and basically should be in int dtype.

In [61]:
# changing dtypes of 4 columns
df = df.withColumn("year",df.year.cast('int'))
df = df.withColumn("frequency",df.frequency.cast('int'))
df = df.withColumn("pages",df.pages.cast('int'))
df = df.withColumn("books",df.books.cast('int'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
# sanity check
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- token: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- frequency: integer (nullable = true)
 |-- pages: integer (nullable = true)
 |-- books: integer (nullable = true)

Let's see if we have any NULL or NaN values in the dataframe...

In [65]:
# Find Count of Null, None, NaN of All DataFrame Columns
# takes a lot of time to run
from pyspark.sql.functions import col,isnan, when, count
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----+---------+-----+-----+
|token|year|frequency|pages|books|
+-----+----+---------+-----+-----+
| 1196| 432|      432|  432|  432|
+-----+----+---------+-----+-----+

Looks like we have null and nan values in all the columns and interestingly we have same number of null and nan values in all the columns except `token` column.

It feels like `year`, `frequency`, `pages` and `books` have nan and null values in the same rows, BUT these things need to be checked. 

---

#### 4.b Create a new DataFrame from a query using Spark SQL, filtering to include only the rows where the token is "data" and describe the new dataset

In [68]:
df.createOrReplaceTempView("google_books")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
# Equivalently my use a SQL query and spark.sql to do the same
df2 =spark.sql('SELECT * FROM google_books WHERE token= "data"')
df2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[token: string, year: int, frequency: int, pages: int, books: int]

In [70]:
# let's see the number of rows now
df2.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

316

So we have just 316 rows which have tokens data.

Let's have a peek into it..

In [71]:
# peeking into the filtered data
df2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----+---------+-----+-----+
|token|year|frequency|pages|books|
+-----+----+---------+-----+-----+
| data|1584|       16|   14|    1|
| data|1614|        3|    2|    1|
| data|1627|        1|    1|    1|
| data|1631|       22|   18|    1|
| data|1637|        1|    1|    1|
| data|1638|        2|    2|    1|
| data|1640|        1|    1|    1|
| data|1642|        1|    1|    1|
| data|1644|        4|    4|    1|
| data|1647|        1|    1|    1|
| data|1651|        1|    1|    1|
| data|1674|        1|    1|    1|
| data|1690|        1|    1|    1|
| data|1693|        1|    1|    1|
| data|1697|        1|    1|    1|
| data|1699|        1|    1|    1|
| data|1700|        1|    1|    1|
| data|1701|       11|   11|    2|
| data|1702|        1|    1|    1|
| data|1703|        1|    1|    1|
+-----+----+---------+-----+-----+
only showing top 20 rows

---

Let's see if we have any null or nan values in this dataset.

In [72]:
# checking null values
df2.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----+---------+-----+-----+
|token|year|frequency|pages|books|
+-----+----+---------+-----+-----+
|    0|   0|        0|    0|    0|
+-----+----+---------+-----+-----+

From above we can see that we have no null or nan values in `df2` (token = "data")

In [74]:
# checking schema
df2.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- token: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- frequency: integer (nullable = true)
 |-- pages: integer (nullable = true)
 |-- books: integer (nullable = true)

---

#### 4.c Write the filtered data back to a directory in the HDFS from Spark using df.write.csv(). Be sure to pass the header=True parameter and examine the contents of what you've written.

In [83]:
df2.write.csv("/user/hadoop/eng_1M_1gram/datatoken.csv", header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
# Read in some data locally
df3 = spark.read.csv("/user/hadoop/eng_1M_1gram2/datatoken.csv", header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
df3.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----+---------+-----+-----+
|token|year|frequency|pages|books|
+-----+----+---------+-----+-----+
| data|1584|       16|   14|    1|
| data|1614|        3|    2|    1|
| data|1627|        1|    1|    1|
| data|1631|       22|   18|    1|
| data|1637|        1|    1|    1|
| data|1638|        2|    2|    1|
| data|1640|        1|    1|    1|
| data|1642|        1|    1|    1|
| data|1644|        4|    4|    1|
| data|1647|        1|    1|    1|
| data|1651|        1|    1|    1|
| data|1674|        1|    1|    1|
| data|1690|        1|    1|    1|
| data|1693|        1|    1|    1|
| data|1697|        1|    1|    1|
| data|1699|        1|    1|    1|
| data|1700|        1|    1|    1|
| data|1701|       11|   11|    2|
| data|1702|        1|    1|    1|
| data|1703|        1|    1|    1|
+-----+----+---------+-----+-----+
only showing top 20 rows

In [81]:
# check
df3.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

316

Bingo! This is what we wanted>>

---

### Thank You!!